In [123]:
import numpy as np
import pandas as pd

In [2]:
ratings_set_df = pd.read_csv("./data_movie_lens_100k/ratings_all_development_set.csv")
users_df = pd.read_csv("./data_movie_lens_100k/user_info.csv")
movies_df = pd.read_csv("./data_movie_lens_100k/movie_info.csv")

In [128]:
print(ratings_set_df)
print(users_df)
#print(movies_df)

       user_id  item_id  rating
0          772       36       3
1          471      228       5
2          641      401       4
3          312       98       4
4           58      504       5
...        ...      ...     ...
89987      415      813       4
89988      842      120       3
89989      574      505       2
89990      757      472       5
89991      503      204       3

[89992 rows x 3 columns]
     user_id  age  is_male  orig_user_id
0          0   24        1             1
1          1   53        0             2
2          2   23        1             3
3          3   24        1             4
4          4   33        0             5
..       ...  ...      ...           ...
938      938   26        0           939
939      939   32        1           940
940      940   20        1           941
941      941   48        0           942
942      942   22        1           943

[943 rows x 4 columns]


In [139]:
from surprise import Dataset, NormalPredictor, Reader, SVD, accuracy

# svd = SVD(
#     n_factors = 10,
#     n_epochs=50
# )

reader = Reader(
    line_format='user item rating', sep=',',
    rating_scale=(1, 5), skip_lines=1)
ratings_df = Dataset.load_from_df(ratings_set_df[['user_id', 'item_id','rating']], reader=reader)

trainset = ratings_df.build_full_trainset()

for n_factors in [1]:
    ## Fit model like our M3
    model = SVD(n_factors=n_factors)
    model.fit(trainset)

    print("global mean:")
    print(model.trainset.global_mean)
    print("shape of bias_per_item: ")
    print(model.bi.shape)
    print("shape of bias_per_user: ")
    print(model.bu.shape)
    print("shape of U (per user vectors): ")
    print(model.pu.shape)
    print("shape of V (per item vectors): ")
    print(model.qi.shape)

user_vectors = model.pu
item_vectors = model.qi

user_features = users_df[['age', 'is_male']].values
item_features = movies_df[['release_year']].values


user_vectors = np.resize(user_vectors, (item_features.shape[0], user_vectors.shape[1]))
item_vectors = np.resize(item_vectors, (item_features.shape[0], item_vectors.shape[1]))

user_features = np.resize(user_features, (item_features.shape[0], user_features.shape[1]))

print(user_vectors.shape)
print(item_vectors.shape)
print(user_features)

print(item_features.shape)

feature_vectors = np.concatenate((user_vectors, item_vectors, user_features, item_features), axis=1)
print(feature_vectors)




global mean:
3.529480398257623
shape of bias_per_item: 
(1662,)
shape of bias_per_user: 
(943,)
shape of U (per user vectors): 
(943, 1)
shape of V (per item vectors): 
(1662, 1)
(1681, 1)
(1681, 1)
[[24  1]
 [53  0]
 [23  1]
 ...
 [48  0]
 [30  1]
 [35  1]]
(1681, 1)
[[-2.54234394e-01  2.05306086e-01  2.40000000e+01  1.00000000e+00
   1.99500000e+03]
 [ 3.69645014e-01  2.74695874e-02  5.30000000e+01  0.00000000e+00
   1.99500000e+03]
 [ 1.89382591e-01  1.08163639e-01  2.30000000e+01  1.00000000e+00
   1.99500000e+03]
 ...
 [ 1.17855106e-01  6.42199984e-02  4.80000000e+01  0.00000000e+00
   1.99800000e+03]
 [-1.35245246e-01 -1.17324656e-02  3.00000000e+01  1.00000000e+00
   1.99400000e+03]
 [-2.21592799e-02 -2.00965414e-01  3.50000000e+01  1.00000000e+00
   1.99600000e+03]]


In [127]:
ratings = ratings_set_df['rating'].values

labels = (ratings >= 4.5).astype(int)

SEED = 12345

X_train, X_test, y_train, y_test = train_test_split(feature_vectors, labels, test_size=0.25, random_state=SEED)

(89992,)


ValueError: Found input variables with inconsistent numbers of samples: [1681, 89992]